## Build a Crew to Tailor Job Applications

In this lesson, you will built your first multi-agent system.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [3]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of embedchain[github,youtube] to determine which version is compatible with other requirements. This could take a while.


In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Task, Crew

In [6]:
import os
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get("OPENAI_API_KEY")
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o-mini'
os.environ["SERPER_API_KEY"] = userdata.get("SERPER_API_KEY")

## crewAI Tools

In [8]:
from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  MDXSearchTool,
  SerperDevTool
)

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path='./fake_resume.md')
semantic_search_resume = MDXSearchTool(mdx='/content/fake_resume.md')

Inserting batches in chromadb: 100%|██████████| 1/1 [00:01<00:00,  1.52s/it]


- Uncomment and run the cell below if you wish to view `fake_resume.md` in the notebook.

In [9]:
from IPython.display import Markdown, display
display(Markdown("./fake_resume.md"))

# Noah Williams
- Email: noah.williams@example.dev
- Phone: +44 11 111 11111

## Profile
Noah Williams is a distinguished Software Engineering Leader with an
18-year tenure in the technology industry, where he has excelled in
leading both remote and in-office engineering teams. His expertise
spans across software development, process innovation, and enhancing
team collaboration. He is highly proficient in programming languages
such as Ruby, Python, JavaScript, TypeScript, and Elixir, alongside
deep expertise in various front end frameworks. Noah's significant
experience in data science and machine learning has enabled him to
spearhead successful deployments of scalable AI solutions and
innovative data model development.

## Work History

### DataKernel: Director of Software Engineering (remote) - 2022 -
Present

- Noah has transformed the engineering division into a key revenue
pillar for DataKernel, rapidly expanding the customer base from
inception to a robust community.
- He spearheaded the integration of cutting-edge AI technologies and
the use of scalable vector databases, which significantly enhanced
the product's capabilities and market positioning.
- Under his leadership, the team has seen a substantial growth in
skill development, with a focus on achieving strategic project goals
that have significantly influenced the company's direction.
- Noah also played a critical role in defining the company's long-
term strategic initiatives, particularly in adopting AI technologies
that have set new benchmarks within the industry.

## Creating Agents

In [10]:
# Agent 1: Researcher
researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on "
         "job posting to help job applicants",
    tools = [scrape_tool, search_tool],
    verbose=True,
    backstory=(
        "As a Job Researcher, your prowess in "
        "navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."
    )
)

In [11]:
# Agent 2: Profiler
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do increditble research on job applicants "
         "to help them stand out in the job market",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    )
)

In [12]:
# Agent 3: Resume Strategist
resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make a "
         "resume stand out in the job market.",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
    )
)

In [13]:
# Agent 4: Interview Preparer
interview_preparer = Agent(
    role="Engineering Interview Preparer",
    goal="Create interview questions and talking points "
         "based on the resume and job requirements",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Your role is crucial in anticipating the dynamics of "
        "interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, "
        "ensuring they can confidently address all aspects of the "
        "job they are applying for."
    )
)

## Creating Tasks

In [14]:
# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences."
    ),
    agent=researcher,
    async_execution=True
)

In [15]:
# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using the GitHub ({github_url}) URLs, and personal write-up "
        "({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=True
)

- You can pass a list of tasks as `context` to a task.
- The task then takes into account the output of those tasks in its execution.
- The task will not run until it has the output(s) from those tasks.

In [16]:
# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume even but "
        "don't make up any information. Update every section, "
        "inlcuding the initial summary, work experience, skills, "
        "and education. All to better reflrect the candidates "
        "abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file="tailored_resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist
)

In [17]:
# Task for Interview Preparer Agent: Develop Interview Materials
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion "
        "points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume "
        "and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)


## Creating the Crew

In [18]:
job_application_crew = Crew(
    agents=[researcher,
            profiler,
            resume_strategist,
            interview_preparer],

    tasks=[research_task,
           profile_task,
           resume_strategy_task,
           interview_preparation_task],

    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [19]:
job_application_inputs = {
    'job_posting_url': 'https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund',
    'github_url': 'https://github.com/joaomdmoura',
    'personal_writeup': """Noah is an accomplished Software
    Engineering Leader with 18 years of experience, specializing in
    managing remote and in-office teams, and expert in multiple
    programming languages and frameworks. He holds an MBA and a strong
    background in AI and data science. Noah has successfully led
    major tech initiatives and startups, proving his ability to drive
    innovation and growth in the tech industry. Ideal for leadership
    roles that require a strategic and innovative approach."""
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [20]:
### this execution will take a few minutes to run
result = job_application_crew.kickoff(inputs=job_application_inputs)

 [DEBUG]: == Working Agent: Tech Job Researcher
 [INFO]: == Starting Task: Analyze the job posting URL provided (https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund) to extract key skills, experiences, and qualifications required. Use the tools to gather content and identify and categorize the requirements.
 [DEBUG]: == [Tech Job Researcher] Task output: 


 [DEBUG]: == Working Agent: Personal Profiler for Engineers
 [INFO]: == Starting Task: Compile a detailed personal and professional profile using the GitHub (https://github.com/joaomdmoura) URLs, and personal write-up (Noah is an accomplished Software
    Engineering Leader with 18 years of experience, specializing in
    managing remote and in-office teams, and expert in multiple
    programming languages and frameworks. He holds an MBA and a strong
    background in AI and data science. Noah has successfully led
    major tech initiatives and startups, proving his abili

 

Relevant Content:
# Noah Williams
- Email: noah.williams@example.dev
- Phone: +44 11 111 11111

## Profile
Noah Williams is a distinguished Software Engineering Leader with an
18-year tenure in the technology industry, where he has excelled in
leading both remote and in-office engineering teams. His expertise
spans across software development, process innovation, and enhancing
team collaboration. He is highly proficient in programming languages
such as Ruby, Python, JavaScript, TypeScript, and Elixir, alongside
deep expertise in various front end frameworks. Noah's significant
experience in data science and machine learning has enabled him to
spearhead successful deployments of scalable AI solutions and
innovative data model development.

## Work History

### DataKernel: Director of Software Engineering (remote) - 2022 -
Present

- Noah has transformed the engineering division into a key revenue
pillar for DataKernel, rapidly expanding the customer base from
inception to a robust co

- Dislplay the generated `tailored_resume.md` file.

In [21]:
from IPython.display import Markdown, display
display(Markdown("/content/tailored_resume.md"))

# João Moura
- **Email:** joao.moura@example.com
- **Phone:** +1 (123) 456-7890
- **LinkedIn:** [linkedin.com/in/joaomoura](https://linkedin.com/in/joaomoura)

## Profile
Dynamic and results-driven Full Stack Engineer with over 18 years of extensive experience in software engineering, specializing in building scalable software solutions. Proven ability to lead teams in developing applications that enhance user experience while ensuring robust back-end functionality. Expertise in a variety of programming languages and frameworks, including JavaScript, Python, Java, React, Angular, and Node.js. Adept in collaborating with cross-functional teams to deliver high-quality products on time and within budget.

## Skills
- **Programming Languages:** JavaScript, Python, Java, Ruby, Elixir
- **Frameworks:** React, Angular, Node.js, Express, Phoenix
- **Databases:** MySQL, MongoDB, PostgreSQL
- **Front-End Technologies:** HTML5, CSS3, Bootstrap, jQuery
- **Development Practices:** Agile methodologies, DevOps, Continuous Integration/Continuous Deployment (CI/CD)
- **Tools:** Git, Docker, AWS, Jenkins
- **UI/UX Design Principles:** Strong understanding of user-centered design processes and methodologies

## Professional Experience

### Senior Full Stack Engineer
**AI Solutions Company** | Remote | 2021 - Present
- Developed and maintained scalable web applications using React and Node.js, improving application performance by 40%.
- Collaborated with UI/UX designers to create user-friendly interfaces, enhancing user engagement and satisfaction.
- Implemented RESTful APIs and integrated third-party services, streamlining data flow and functionality.

### Full Stack Developer
**Tech Innovations Ltd.** | Porto, Portugal | 2018 - 2021
- Led the development of a multi-tenant SaaS application using Angular and Express, which increased customer retention by 30%.
- Managed database design and optimization for MySQL and MongoDB, ensuring efficient data retrieval and storage.
- Trained and mentored junior developers, fostering a collaborative team environment.

### Software Engineer
**Creative Solutions** | Lisbon, Portugal | 2014 - 2018
- Designed and implemented complex software solutions, leading to a 25% reduction in development time through the use of reusable components.
- Conducted code reviews and maintained coding standards, resulting in improved code quality and reduced bugs by 15%.

## Education
**Bachelor of Science in Computer Engineering**  
University of Porto, Portugal | 2002 - 2006

## Certifications
- **AWS Certified Solutions Architect**
- **Certified ScrumMaster (CSM)**

## Additional Information
- Fluent in English and Portuguese.
- Passionate about open-source projects and contributing to community-driven software development initiatives.
- Speaker at various tech conferences and workshops focused on software development and best practices.

---

This resume emphasizes João Moura's extensive background in full-stack development and aligns closely with the requirements for the Full Stack Engineer position at AI Fund. It showcases his technical expertise, leadership skills, and commitment to delivering high-quality software solutions.

- Dislplay the generated `interview_materials.md` file.

In [22]:
display(Markdown("/content/interview_materials.md"))

### Interview Questions and Talking Points for João Moura

#### 1. General Experience
- **Question**: Can you tell us about your experience as a Full Stack Engineer and how it relates to the responsibilities outlined in this position?
  - **Talking Point**: João has over 18 years of experience in software engineering, specializing in building scalable software solutions. His extensive background aligns perfectly with the requirement to produce scalable software solutions for portfolio companies at AI Fund.

#### 2. Collaboration and Teamwork
- **Question**: Describe a time when you collaborated with product managers and development teams to create a software solution. What was your role, and what was the outcome?
  - **Talking Point**: João has led diverse teams both remotely and in-office, emphasizing collaboration. His role in developing the CrewAI framework involved community feedback and cross-functional teamwork, showcasing his ability to work effectively with various stakeholders.

#### 3. Front-End Development
- **Question**: What front-end technologies have you used in your previous projects, and how do you ensure an appealing visual design?
  - **Talking Point**: João is proficient in front-end technologies such as React, Angular, HTML, CSS, and JavaScript. He has collaborated with UI/UX designers to enhance user engagement and create user-friendly interfaces in his current role at AI Solutions Company.

#### 4. Back-End Development and Database Management
- **Question**: Can you discuss your experience with database management and API development?
  - **Talking Point**: João has managed databases like MySQL and MongoDB and has implemented RESTful APIs to streamline data flow in applications. His experience in optimizing database design has resulted in improved application performance.

#### 5. Testing and Debugging
- **Question**: What approaches do you take to test software for responsiveness and efficiency?
  - **Talking Point**: João follows Agile methodologies and emphasizes testing as part of the development process. His experience in troubleshooting and debugging software has led to significant improvements in performance and user satisfaction.

#### 6. Security and Data Protection
- **Question**: How do you approach creating security and data protection settings in your applications?
  - **Talking Point**: João understands the importance of security in software development and has implemented security measures in his projects to protect user data and ensure compliance with regulations.

#### 7. Adaptability and Innovation
- **Question**: How do you approach challenges in an early-stage environment, and can you provide an example?
  - **Talking Point**: João thrives in early-stage environments, as demonstrated by his founding of CrewAI. He has shown grit and creativity in overcoming challenges and driving innovation, which aligns with AI Fund's desired characteristics of accountability and ownership orientation.

#### 8. Communication Skills
- **Question**: How do you ensure effective communication with both technical and non-technical stakeholders?
  - **Talking Point**: João is known for his clear and concise communication style, which fosters collaboration and transparency. His ability to explain complex technical concepts to non-technical stakeholders has been key to his success in various roles.

#### 9. Interest in AI/ML
- **Question**: Can you elaborate on your experience or interest in AI and machine learning?
  - **Talking Point**: With a strong background in AI frameworks and data science, João is passionate about leveraging AI technologies. His work on AI-related projects positions him well to contribute to AI Fund's goals.

#### 10. Professional Development
- **Question**: How do you stay updated with current technologies and trends in software development?
  - **Talking Point**: João actively engages in open-source projects and tech communities, which helps him stay informed about the latest trends and best practices. His commitment to continuous learning is evident in his participation in workshops and tech conferences.

### Conclusion
João Moura's extensive experience, technical skills, and commitment to collaboration and innovation make him a strong candidate for the Full Stack Engineer position at AI Fund. He should focus on highlighting these aspects during his interview to demonstrate his alignment with the company's needs and values.